In [1]:
import torch 

#### Data loading 

In [2]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]  # target label 
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]  # input data 
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

t_un = 0.1 * t_u    # uniformed input data between[-10.0, 10.0]
print(t_un)

tensor([3.5700, 5.5900, 5.8200, 8.1900, 5.6300, 4.8900, 3.3900, 2.1800, 4.8400,
        6.0400, 6.8400])


#### Design your model 
* model define 
* initialize the parameters and instantiate your model 

In [3]:
def model(t_u:torch.Tensor, w:torch.Tensor, b:torch.Tensor):
    return w * t_u + b

In [4]:
# _Start: initialize the parameters 
w = torch.ones(())
b = torch.zeros(())
# _End: initialize the parameters 


# _Start: instantiate a model with initial values 
t_p = model(t_u, w, b)
# _End: instantiate a model with initial values 

#### Loss function : MSE

In [5]:
def loss_fn(t_p:torch.Tensor, t_c:torch.Tensor) -> torch.Tensor:  # MSE 
    squared_diffs = (t_p - t_c) ** 2 
    return squared_diffs.mean() 

***

# 1. Autograd 
* set ```requires_grad = True ``` for using <b>Autograd</b>

In [6]:
params = torch.tensor([1.0, 0.1], requires_grad=True)

print("Initial PyTorch tensors' \'grad\' is None: ", params.grad is None)

Initial PyTorch tensors' 'grad' is None:  True


In [7]:
loss = loss_fn( model(t_u, *params), t_c)

loss.backward() # backpropagation for derivative 

print("derivatives of the loss with respect to each element of params: ",params.grad)

derivatives of the loss with respect to each element of params:  tensor([4527.6567,   82.8000])


### Set zero gradient 
* Before computing the gradients, it should be set zero 
    > Use ```.zero_()``` method

In [8]:
if params.grad is not None: 
    params.grad.zero_() 
    
print("derivatives of the loss with respect to each element of params: ",params.grad)

derivatives of the loss with respect to each element of params:  tensor([0., 0.])


# 2. training loop

In [9]:
def training_loop(n_epochs:int, learning_rate:float , params:torch.Tensor, t_u:torch.Tensor, t_c:torch.Tensor) -> torch.Tensor:
    
    # _Start: training loop for n_poches
    for epoch in range(1, n_epochs + 1):
        
        # _Start: set zero gradient
        if params.grad is not None:
            params.grad.zero_()
        # _End: set zero gradient

        
        # _Start: model prediction and estimate the loss 
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)
        # _End: model prediction and estimate the loss 
        
        
        # _Start: backpropagate and update the parameters
        loss.backward()
        with torch.no_grad():
            params -= learning_rate * params.grad
        # _End: backpropagate and update the parameters
            
        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))
    
    # _End: training loop for n_poches
    return params

In [10]:
training_loop(
    n_epochs = 5000,
    learning_rate = 1e-2,
    params = torch.tensor([1.0, 0.0], requires_grad=True),
    t_u = t_un,
    t_c = t_c)

Epoch 500, Loss 7.860116
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957697
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

# 3. training loop with optimizers 
* setting hyperparameters 

In [11]:
import torch.optim as optim 
dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'AdamW',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

In [12]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
t_p = model(t_un, *params)

### optimizer's working test 
* ```optimizer.zero_grad()```
* ```loss.backward()```
* ```optimizer.step()```

In [13]:
print("Before update: ", params )

Before update:  tensor([1., 0.], requires_grad=True)


In [14]:
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

loss = loss_fn(t_p, t_c)

optimizer.zero_grad() 
loss.backward()
optimizer.step()

print("After update: ", params)

After update:  tensor([1.7761, 0.1064], requires_grad=True)


### So, Let's define the new training loop 

In [15]:
def training_loop_with_optim(n_epochs:int, params:torch.Tensor, t_u:torch.Tensor, t_c: torch.Tensor, optimizer) -> torch.Tensor:
    
    # _Start: training loop for n_poches
    for epoch in range(1, n_epochs + 1):
        
        t_p = model(t_u, *params)
        loss = loss_fn(t_p, t_c)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print('Epoch %d, Loss %f' % (epoch, float(loss)))

    # _End: training loop for n_poches
    return params

In [17]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2 
optimizer = optim.SGD([params], lr=learning_rate)


training_loop_with_optim(
    n_epochs = 5000,
    params = params,
    t_u = t_un,
    t_c = t_c,
    optimizer = optimizer)

Epoch 500, Loss 7.860116
Epoch 1000, Loss 3.828538
Epoch 1500, Loss 3.092191
Epoch 2000, Loss 2.957697
Epoch 2500, Loss 2.933134
Epoch 3000, Loss 2.928648
Epoch 3500, Loss 2.927830
Epoch 4000, Loss 2.927679
Epoch 4500, Loss 2.927652
Epoch 5000, Loss 2.927647


tensor([  5.3671, -17.3012], requires_grad=True)

# 4. training loop with validation set 

In [22]:
print(t_u.shape)
print(t_u.shape[0])

n_samples = t_u.shape[0]
print("the total number of samples: ", n_samples)

torch.Size([11])
11
the total number of samples:  11


In [23]:
n_val = int(0.2 * n_samples)
print("the number of validation: ", n_val)

the number of validation:  2


In [24]:
shuffled_indices = torch.randperm(n_samples)
print(shuffled_indices)

tensor([ 7,  5,  4,  1,  6,  8,  9,  0,  2,  3, 10])


In [25]:
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [28]:
train_t_u = t_u[train_indices]   # input for train 
train_t_c = t_c[train_indices]   # label for train 

val_t_u = t_u[val_indices]       # input for validation 
val_t_c = t_c[val_indices]       # label for validation 

train_t_un = 0.1 * train_t_u     # uniformed 
val_t_un = 0.1 * val_t_u

In [29]:
def training_loop_with_validation(n_epochs:int, optimizer, params:torch.Tensor, train_t_u, val_t_u, train_t_c, val_t_c):
    
    # _Start: training loop for n_poches
    for epoch in range(1, n_epochs + 1):
        
        # _Start: for training set 
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        # _End: for training set 
        
        # _Start: for validation set 
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)
        # End: for validation set 
    
        # _Start: backpropagation and update 
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # _End: backpropagation and update 

        if epoch <= 3 or epoch % 500 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(train_loss), float(val_loss)))
    
    # _End: training loop for n_poches
    return params

In [30]:
params = torch.tensor([1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)


In [31]:
training_loop_with_validation(
    n_epochs = 3000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un,
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 32.34067916870117, Validation loss 296.4708251953125
Epoch 2, Training loss 21.572343826293945, Validation loss 206.237548828125
Epoch 3, Training loss 18.682376861572266, Validation loss 166.02215576171875
Epoch 500, Training loss 6.659817695617676, Validation loss 57.58177185058594
Epoch 1000, Training loss 3.6122732162475586, Validation loss 31.565763473510742
Epoch 1500, Training loss 2.776271104812622, Validation loss 21.065752029418945
Epoch 2000, Training loss 2.5469415187835693, Validation loss 16.423871994018555
Epoch 2500, Training loss 2.4840309619903564, Validation loss 14.227890014648438
Epoch 3000, Training loss 2.4667739868164062, Validation loss 13.142255783081055


tensor([  4.7232, -14.6063], requires_grad=True)

# Edit: autograd for validation is no needed 
* Actually, we don't need to backpropagate for validation set 
* So, switch off autograd for validation. 
    > for escaping accumulated gradients.

In [32]:
def training_loop_edit(n_epochs:int, optimizer, params:torch.Tensor, train_t_u, val_t_u, train_t_c, val_t_c):
    
    # _Start: training loop for n_poches
    for epoch in range(1, n_epochs + 1):
        
        # _Start: for training set 
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
        # _End: for training set
        
        # _Start: for validation set 
        with torch.no_grad():
            val_t_p = model(val_t_u, *params)
            val_loss = loss_fn(val_t_p, val_t_c)
            assert val_loss.requires_grad == False
        # _End: for validation set 
            
        # _Start: backpropagation and update 
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # _End: backpropagation and update 
        
        if epoch <= 3 or epoch % 500 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(train_loss), float(val_loss)))
    
    # _End: training loop for n_poches
    return params

In [34]:
training_loop_edit(
    n_epochs = 3000,
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un,
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 2.4667561054229736, Validation loss 13.140748977661133
Epoch 2, Training loss 2.466740369796753, Validation loss 13.139253616333008
Epoch 3, Training loss 2.4667227268218994, Validation loss 13.137746810913086
Epoch 500, Training loss 2.4620394706726074, Validation loss 12.591374397277832
Epoch 1000, Training loss 2.460740804672241, Validation loss 12.30767822265625
Epoch 1500, Training loss 2.4603841304779053, Validation loss 12.160470962524414
Epoch 2000, Training loss 2.4602880477905273, Validation loss 12.083722114562988
Epoch 2500, Training loss 2.4602601528167725, Validation loss 12.043630599975586
Epoch 3000, Training loss 2.460251808166504, Validation loss 12.022647857666016


tensor([  4.7843, -14.9109], requires_grad=True)